In [3]:
import os.path as op
import os
import pandas as pd
import numpy as np
from atlasreader import create_output

zmaps_folder = '/home/hyruuk/GitHub/neuromod/shinobi_fmri/data/processed/z_maps/subject-level/'

## Add atlas to cluster table

In [3]:
annotations = os.listdir(zmaps_folder)
for annot in annotations:
    zmaps_list = os.listdir(op.join(zmaps_folder, annot))
    for zmap in zmaps_list:
        if '.nii.gz' in zmap:
            print(zmap)
            filepath = op.join(zmaps_folder, annot, zmap)
            create_output(filepath, cluster_extent=5, voxel_thresh=3, direction='both')

sub-04_simplemodel_HealthLoss.nii.gz
sub-02_simplemodel_HealthLoss.nii.gz
sub-01_simplemodel_HealthLoss.nii.gz
sub-06_simplemodel_HealthLoss.nii.gz
sub-04_simplemodel_UP.nii.gz
sub-02_simplemodel_UP.nii.gz
sub-06_simplemodel_UP.nii.gz
sub-01_simplemodel_UP.nii.gz
sub-06_simplemodel_JUMP.nii.gz
sub-04_simplemodel_JUMP.nii.gz
sub-02_simplemodel_JUMP.nii.gz
sub-01_simplemodel_JUMP.nii.gz
sub-02_simplemodel_HIT.nii.gz
sub-04_simplemodel_HIT.nii.gz
sub-01_simplemodel_HIT.nii.gz
sub-06_simplemodel_HIT.nii.gz
sub-04_simplemodel_DOWN.nii.gz
sub-06_simplemodel_DOWN.nii.gz
sub-02_simplemodel_DOWN.nii.gz
sub-01_simplemodel_DOWN.nii.gz
sub-04_simplemodel_LEFT.nii.gz
sub-02_simplemodel_LEFT.nii.gz
sub-01_simplemodel_LEFT.nii.gz
sub-06_simplemodel_LEFT.nii.gz
sub-06_simplemodel_Kill.nii.gz
sub-04_simplemodel_Kill.nii.gz
sub-01_simplemodel_Kill.nii.gz
sub-02_simplemodel_Kill.nii.gz
sub-04_simplemodel_RIGHT.nii.gz
sub-02_simplemodel_RIGHT.nii.gz
sub-06_simplemodel_RIGHT.nii.gz
sub-01_simplemodel_RIGHT

In [4]:
# Format clustertable
import glob
tables_list = glob.glob(op.join(zmaps_folder, '*', '*clusters.csv'))

tables_df = []
for table in tables_list:
    df = pd.read_csv(table)
    df['annotation'] = table.split('/')[-2]
    df['subject'] = table.split('/')[-1].split('_')[0]
    df = df.sort_values(by='cluster_mean').tail(3) # Select 3 biggest clusters
    tables_df.append(df)
tables_df = pd.concat(tables_df, ignore_index=True).astype(str).drop(columns=['desikan_killiany', 'harvard_oxford'])
tables_df = tables_df[tables_df['annotation'].isin(['JUMP', 'HIT', 'Kill', 'HealthLoss'])]

def concatenate_strings(series):
    return "; ".join(series)
agg_dict={}
for col in tables_df.columns:
    if col not in(['annotation', 'subject']):
        agg_dict[col] = concatenate_strings
tables_df_grouped = tables_df.groupby(['annotation', 'subject']).agg(agg_dict)

tables_df_grouped['cluster (id / x / y / z / mean / volume)'] = tables_df_grouped['cluster_id'] + ' / ' + tables_df_grouped['peak_x'] + ' / ' + tables_df_grouped['peak_y'] + ' / ' + tables_df_grouped['peak_z'] + ' / ' + tables_df_grouped['cluster_mean'] + ' / ' + tables_df_grouped['volume_mm']
tables_df_grouped.drop(columns=['cluster_id', 'peak_x', 'peak_y', 'peak_z', 'cluster_mean', 'volume_mm'], inplace=True)
tables_df_grouped.to_csv('/home/hyruuk/GitHub/neuromod/shinobi_fmri/reports/tables/cluster_tables_extent-5_thresh-3_pos.csv')

#tables_df_grouped['aal'] = tables_df_grouped['aal'].str.wrap(30)
tables_df_grouped

aal  \
annotation subject                                                      
HIT        sub-01   33.22% Parietal_Sup_L; 21.81% Precuneus_R; 15....   
           sub-02   100.00% Occipital_Mid_L; 85.00% Occipital_Mid_...   
           sub-04   100.00% Thalamus_L; 8.52% no_label; 100.00% no...   
           sub-06   27.56% Postcentral_L; 16.99% no_label; 14.26% ...   
HealthLoss sub-01   49.40% Frontal_Sup_Medial_L; 21.69% Cingulate_...   
           sub-02   100.00% Precuneus_R; 100.00% Angular_R; 84.62%...   
           sub-04   41.67% Temporal_Sup_L; 41.67% no_label; 16.67%...   
           sub-06   83.33% Temporal_Mid_L; 16.67% Occipital_Mid_L;...   
JUMP       sub-01   12.07% Occipital_Sup_L; 11.46% Parietal_Sup_L;...   
           sub-02   85.71% Postcentral_L; 14.29% Precentral_L; 100...   
           sub-04   50.00% Precuneus_L; 50.00% no_label; 9.95% Par...   
           sub-06   44.05% SupraMarginal_L; 27.38% no_label; 20.24...   
Kill       sub-01   16.03% Occipital_Mid_L; 11.43% Calcarine_R; 10...   
           sub-02   100.00% Calcarine_R; 60.00% Insula_R; 20.00% F...   
           sub-04   75.00% Fusiform_R; 12.50% Lingual_R; 12.50% Pa...   
           sub-06   45.51% Frontal_Mid_2_L; 33.97% Frontal_Sup_2_L...   

                             cluster (id / x / y / z / mean / volume)  
annotation subject                                                     
HIT        sub-01   3; 44; 40 / 11.5; 15.5; 15.5 / -44.5; -64.5; -...  
           sub-02   50; 20; 60 / -28.5; 31.5; 7.5 / -76.5; -88.5; ...  
           sub-04   38; 1; 34 / -8.5; -64.5; -12.5 / -16.5; -16.5;...  
           sub-06   1; 14; 5 / -64.5; -48.5; -40.5 / -60.5; -28.5;...  
HealthLoss sub-01   3; 2; 28 / -4.5; 67.5; -48.5 / 67.5; -32.5; 43...  
           sub-02   15; 20; 5 / 11.5; 47.5; -0.5 / -56.5; -72.5; -...  
           sub-04   11; 24; 6 / -64.5; 7.5; -52.5 / -44.5; -84.5; ...  
           sub-06   10; 8; 2 / -40.5; 51.5; -44.5 / -68.5; -44.5; ...  
JUMP       sub-01   1; 40; 19 / -12.5; 67.5; 43.5 / -48.5; -12.5; ...  
           sub-02   19; 22; 5 / -36.5; 31.5; 55.5 / -20.5; -12.5; ...  
           sub-04   30; 1; 20 / -16.5; -28.5; 15.5 / -44.5; -60.5;...  
           sub-06   7; 9; 3 / -56.5; 59.5; -48.5 / -24.5; -36.5; -...  
Kill       sub-01   1; 20; 4 / 23.5; 23.5; -16.5 / -96.5; 35.5; -5...  
           sub-02   24; 43; 48 / 11.5; 31.5; 15.5 / -68.5; 27.5; -...  
           sub-04   9; 10; 1 / 27.5; 35.5; -24.5 / -44.5; 15.5; -6...  
           sub-06   3; 7; 19 / -28.5; -52.5; 59.5 / -0.5; 19.5; -1...

In [159]:
# Create a DF that shows which regions comes up the most in each annotation

labels_df = {'label':[], 'annotation':[], 'subject':[]}
for idx, content in tables_df_grouped.iterrows():
    for label in content['aal'].split('; '):
        label_clean = label.split('%')[1]
        labels_df['label'].append(label_clean)
        labels_df['annotation'].append(idx[0])
        labels_df['subject'].append(idx[1])
labels_df = pd.DataFrame(labels_df)

label_list, count_list = np.unique(labels_df['label'], return_counts=True)

occurence_df = {'label':[], 'occurences':[], 'present in annotation':[], 'present in subject':[]}

for idx, label in enumerate(label_list):
    occurence_df['label'].append(label)
    occurence_df['occurences'].append(count_list[idx])
    present_string = ''
    for annotation in np.unique(labels_df['annotation']):
        if label in labels_df[labels_df['annotation']==annotation]['label'].unique():
            present_string += annotation + '; '
    occurence_df['present in annotation'].append(present_string)
    present_string = ''
    for subject in np.unique(labels_df['subject']):
        if label in labels_df[labels_df['subject']==subject]['label'].unique():
            present_string += subject + '; '
    occurence_df['present in subject'].append(present_string)

occurence_df = pd.DataFrame(occurence_df).sort_values(by='occurences', ascending=False).reset_index(drop=True)
# Save df as csv
occurence_df.to_csv('/home/hyruuk/GitHub/neuromod/shinobi_fmri/reports/tables/occurence_df_extent-10_thresh-3.csv', index='label')

occurence_df


,label,occurences,present in annotation,present in subject
0,no_label,20,HIT; HealthLoss; JUMP; Kill;,sub-01; sub-02; sub-04; sub-06;
1,Precuneus_L,8,HIT; JUMP; Kill;,sub-01; sub-02; sub-06;
2,Postcentral_L,7,HIT; JUMP; Kill;,sub-01; sub-02; sub-04; sub-06;
3,Temporal_Mid_R,7,HIT; HealthLoss; JUMP; Kill;,sub-01; sub-02; sub-06;
4,Precentral_L,7,HIT; JUMP; Kill;,sub-01; sub-02; sub-04; sub-06;
5,Occipital_Mid_L,7,HIT; HealthLoss; JUMP; Kill;,sub-01; sub-02; sub-06;
6,Precuneus_R,6,HIT; JUMP; Kill;,sub-01; sub-02; sub-06;
7,Temporal_Mid_L,6,HIT; HealthLoss; JUMP;,sub-01; sub-02; sub-06;
8,Parietal_Sup_L,5,HIT; JUMP; Kill;,sub-01; sub-04; sub-06;
9,Parietal_Sup_R,5,HIT; JUMP; Kill;,sub-01; sub-04; sub-06;
